## 结构化输出 
### BaseModel 
### TypedDict

In [ ]:
# 存在问题，小模型不稳定，可能正确输出，可能出错。
# How to return structured data from a model
# Pydantic class
from typing import Optional
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

llm = ChatOpenAI(
    model="Qwen/Qwen3-8B",
    # model="Qwen/Qwen3-30B-A3B-Instruct-2507",
    api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
    base_url="https://api.siliconflow.cn/v1/",
    # temperature=0,
    )

# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

structured_llm = llm.with_structured_output(Joke)
msg = structured_llm.invoke("Tell me a joke about cats")
msg

Joke(setup="Why don't cats ever get cold?", punchline="Why don't cats ever get cold? Because they always have nine lives and a warm heart!", rating=3)

In [2]:
# TypedDict or JSON Schema
from typing import Optional
from typing_extensions import Annotated, TypedDict

# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

msg = structured_llm.invoke("Tell me a joke about cats")
msg

{'punchline': 'Why did the cat get kicked out of the house?',
 'setup': 'Because it kept knocking things over and making a mess with its paws.'}

In [ ]:
# json_schema
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

msg = structured_llm.invoke("Tell me a joke about cats")
msg


{'punchline': "Why don't cats make good dancers? They always end up in the spotlight, but never in the rhythm!",
 'setup': "I told my cat I'd take him to the vet, and he said, 'You're gonna take me to the vet? That's the third time this week!'"}

In [6]:
# Choosing between multiple schemas
from typing import Union


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")


class FinalResponse(BaseModel):
    final_output: Union[Joke, ConversationalResponse]


structured_llm = llm.with_structured_output(FinalResponse)

msg = structured_llm.invoke("Tell me a joke about cats")
msg


FinalResponse(final_output=ConversationalResponse(response="Why don't cats make good secret agents? They always meow their plans to the whole neighborhood!"))

In [7]:
msg = structured_llm.invoke("How are you today?")
msg


FinalResponse(final_output=ConversationalResponse(response="I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help! How can I assist you today?"))

In [ ]:
# Using TypedDict
from typing import Optional, Union
from typing_extensions import Annotated, TypedDict

class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

class ConversationalResponse(TypedDict):
    """Respond in a conversational manner. Be kind and helpful."""

    response: Annotated[str, ..., "A conversational response to the user's query"]


class FinalResponse(TypedDict):
    final_output: Union[Joke, ConversationalResponse]


structured_llm = llm.with_structured_output(FinalResponse).with_retry(3)

msg = structured_llm.invoke("Tell me a joke about cats")
msg

{'fina': "Here's a joke about cats:",
 'texto': "Why don't cats play cards with their owners? Because they're always afraid of the cat's cradle!"}

In [11]:
structured_llm.invoke("How are you today?")

{'final_output': {'response': "I'm just a virtual assistant, but I'm here and ready to help! How can I assist you today?"}}

In [13]:
# Few-shot prompting
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")


{'final_punchline': "Woodpecker, woodpecker... who's there? Woodpecker who? *smacks forehead* I just pecked myself in the head!",
 'rating': 9}

In [14]:
# 当构建输出的底层方法是工具调用时，我们可以将示例作为显式工具调用传入。您可以在 API 参考中检查您使用的模型是否使用了工具调用。
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("Tell me a joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Why don't planes ever get tired?",
                    "punchline": "Because they have rest wings!",
                    "rating": 2,
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
    HumanMessage("Tell me another joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Cargo",
                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",
                    "rating": 10,
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Now about caterpillars", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Caterpillar",
                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",
                    "rating": 5,
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") \
and the final punchline (the response to "<setup> who?")."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})

{'name': 'joke',
 'arguments': {'setup': 'Crocodile',
  'punchline': "Crocodile 'I'm gonna get you!', but I’m already on the plane to a different country!",
  'rating': 8}}

In [15]:
# Specifying the method for structuring outputs
structured_llm = llm.with_structured_output(None, method="json_mode")

structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)

{'setup': "Why don't cats play fair in chess?",
 'punchline': 'They always try to take the queen.'}

In [16]:
structured_llm = llm.with_structured_output(Joke, include_raw=True)

msg = structured_llm.invoke("Tell me a joke about cats")
msg


{'raw': AIMessage(content='{\n  "punchline": "I told my cat I was going to the vet. She meowed.",\n  "setup": "I told my cat I was going to the vet, and she meowed."\n}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 18, 'total_tokens': 63, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198d1789124e25bd8da4afa555799a3', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--ff3b4ce4-9e50-44fd-b330-5e3d3b09fbee-0', usage_metadata={'input_tokens': 18, 'output_tokens': 45, 'total_tokens': 63, 'input_token_details': {}, 'output_token_details': {}}),
 'parsed': {'punchline': 'I told my cat I was going to the vet. She meowed.',
  'setup': 'I told my cat I was going to the vet, and she meowed.'},
 'parsing_error': None}

In [17]:
# 直接提示和解析模型输出
from typing import List
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )

class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]

# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.invoke({"query": query}).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Information about a person.", "properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "height_in_meters": {"description": "The height of the person expressed in meters.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Identifying information about all people in a text.", "properties": {"people": {"items"

In [18]:
chain = prompt | llm | parser

chain.invoke({"query": query})

People(people=[Person(name='Anna', height_in_meters=1.8288)])

In [19]:
# 自定义
import json
import re
from typing import List

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Output your answer as JSON that  "
            "matches the given schema: \`\`\`json\n{schema}\n\`\`\`. "
            "Make sure to wrap the answer in \`\`\`json and \`\`\` tags",
        ),
        ("human", "{query}"),
    ]
).partial(schema=People.model_json_schema())


# Custom parser
def extract_json(message: AIMessage) -> List[dict]:
    """Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.

    Parameters:
        text (str): The text containing the JSON content.

    Returns:
        list: A list of extracted JSON strings.
    """
    text = message.content
    # Define the regular expression pattern to match JSON blocks
    pattern = r"\`\`\`json(.*?)\`\`\`"

    # Find all non-overlapping matches of the pattern in the string
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the list of matched JSON strings, stripping any leading or trailing whitespace
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")

In [20]:
query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.format_prompt(query=query).to_string())

System: Answer the user query. Output your answer as JSON that  matches the given schema: \`\`\`json
{'$defs': {'Person': {'description': 'Information about a person.', 'properties': {'name': {'description': 'The name of the person', 'title': 'Name', 'type': 'string'}, 'height_in_meters': {'description': 'The height of the person expressed in meters.', 'title': 'Height In Meters', 'type': 'number'}}, 'required': ['name', 'height_in_meters'], 'title': 'Person', 'type': 'object'}}, 'description': 'Identifying information about all people in a text.', 'properties': {'people': {'items': {'$ref': '#/$defs/Person'}, 'title': 'People', 'type': 'array'}}, 'required': ['people'], 'title': 'People', 'type': 'object'}
\`\`\`. Make sure to wrap the answer in \`\`\`json and \`\`\` tags
Human: Anna is 23 years old and she is 6 feet tall


In [21]:
chain = prompt | llm | extract_json

chain.invoke({"query": query})

[{'people': [{'name': 'Anna', 'height_in_meters': 1.8288}]}]

In [22]:
# # 与其他工具结合使用
# # 1. Bind tools first
# llm_with_tools = llm.bind_tools([web_search_tool, calculator_tool])
# # 2. Apply structured output
# structured_llm = llm_with_tools.with_structured_output(People)
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


class SearchResult(BaseModel):
    """Structured search result."""

    query: str = Field(description="The search query")
    findings: str = Field(description="Summary of findings")


# Define tools
search_tool = {
    "type": "function",
    "function": {
        "name": "web_search",
        "description": "Search the web for information",
        "parameters": {
            "type": "object",
            "properties": {"query": {"type": "string", "description": "Search query"}},
            "required": ["query"],
        },
    },
}

# Correct approach
# llm = ChatOpenAI()
llm_with_search = llm.bind_tools([search_tool])
structured_search_llm = llm_with_search.with_structured_output(SearchResult)

# Now you can use both search and get structured output
result = structured_search_llm.invoke("Search for latest AI research and summarize")


ValidationError: 1 validation error for SearchResult
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value="Here’s a **summary of ...e in-depth information!", input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid

In [ ]:
result

In [1]:
import json
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
import random
import time

MODEL = "Qwen/Qwen3-30B-A3B-Instruct-2507"

MODELSCOPE_API_KEY_LIST = [
    "ms-e2666046-2f3b-4c76-bcc0-e21f8ebf9ea1",
    "ms-fd864a65-db93-4d86-8495-92f587ee5f74",
    "ms-fb4af56e-d68a-4b3b-94ef-ea6471c8a209",
]

cnt = 0
def llm_modelscope(model: str=MODEL, api_key: Optional[str]=None, late_time: float=3.0, **kwargs):
    if not api_key:
        api_key = random.choice(MODELSCOPE_API_KEY_LIST)
    time.sleep(late_time)
    global cnt
    print(f"model={model} kwargs={kwargs} cnt={cnt}")
    cnt += 1
    llm = ChatOpenAI(
        model=model,
        api_key=api_key,
        base_url="https://api-inference.modelscope.cn/v1/",
        **kwargs,
    )
    
    return llm

In [2]:
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="姓名")
    age: int = Field(description="年龄")

llm = llm_modelscope()

# 先把 response_format 交给模型（模型尽量直接输出合法 JSON）
person_json_format = {
    "type":"json_schema",
    "json_schema": {
        "name": Person.__name__,
        "schema": Person.model_json_schema(),
        "strict": True
    }
}

model=Qwen/Qwen3-30B-A3B-Instruct-2507 kwargs={} cnt=0


In [7]:
structured = llm.with_structured_output(Person.model_json_schema())
person_obj = structured.invoke("输出一个 person JSON 对象（name, age）")
print(person_obj, type(person_obj))

{'name': '张三', 'age': 25} <class 'dict'>


In [8]:
llm_with_resp = llm.bind(response_format=person_json_format)
person_obj = llm_with_resp.invoke("输出一个 person JSON 对象（name, age）")
print(person_obj, type(person_obj))
# {'name': '张三', 'age': 28} <class 'dict'>

content='{\n  "name": "张三",\n  "age": 28\n}' additional_kwargs={'parsed': None, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 21, 'total_tokens': 39, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-30B-A3B-Instruct-2507', 'system_fingerprint': '', 'id': 'chatcmpl-e876f70d-e6bb-47f7-b0b1-724438ea1c28', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--44f2f064-f7b0-4085-9608-ffe530d35de7-0' usage_metadata={'input_tokens': 21, 'output_tokens': 18, 'total_tokens': 39, 'input_token_details': {}, 'output_token_details': {}} <class 'langchain_core.messages.ai.AIMessage'>


In [5]:
llm_with_resp = llm.bind(response_format=person_json_format)
# 再让 LangChain 把返回解析/校验成 Person
structured = llm_with_resp.with_structured_output(Person.model_json_schema())
person_obj = structured.invoke("输出一个 person JSON 对象（name, age）")
print(person_obj, type(person_obj))
# {'name': '张三', 'age': 28} <class 'dict'>

{'name': '张三', 'age': 25} <class 'dict'>


In [7]:
Person.model_json_schema()

{'properties': {'name': {'description': '姓名',
   'title': 'Name',
   'type': 'string'},
  'age': {'description': '年龄', 'title': 'Age', 'type': 'integer'}},
 'required': ['name', 'age'],
 'title': 'Person',
 'type': 'object'}

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser(pydantic_object=Person)
json_parser.invoke('{\n  "name": "Alice",\n  "age": 30\n}')

{'name': 'Alice', 'age': 30}

In [3]:
llm_with_resp.invoke("输出一个 person JSON 对象（name, age）")

AIMessage(content='{\n  "name": "Alice",\n  "age": 30\n}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 21, 'total_tokens': 38, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-30B-A3B-Instruct-2507', 'system_fingerprint': '', 'id': 'chatcmpl-3048aeaf-7169-4396-88c3-c27a76eb293b', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--1fb946fa-9a18-46d7-bc3e-b465efc4e463-0', usage_metadata={'input_tokens': 21, 'output_tokens': 17, 'total_tokens': 38, 'input_token_details': {}, 'output_token_details': {}})

In [4]:
llm.with_structured_output(Person).invoke("输出一个 person JSON 对象（name, age）")

Person(name='张三', age=28)